In [1]:
import sys
import os

# Get the absolute path of the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add it to sys.path if not already added
if project_root not in sys.path:
    sys.path.append(project_root)

In [4]:
from src.functions.data_generation import DataGeneration
from sklearn.model_selection import train_test_split

data = DataGeneration.generate_data(
    n_samples=10000,
    n_features=20,
    n_informative=4,
    relationship='friedman2',
    noise=0.1,
    random_state=58495
)

# Draw bootstrap sample
bootstrap_data = data.sample(n=1000, replace=True, random_state=89321)
bootstrap_data = DataGeneration.add_epsilon(bootstrap_data)

display(bootstrap_data)

X = bootstrap_data.drop(columns=["Target"])
y = bootstrap_data["Target"]

# Split data
X_proper, X_test, y_proper, y_test = train_test_split(X, y, test_size=0.5)

        
def to_numpy_safe(x):
    return x.to_numpy() if hasattr(x, "to_numpy") else x

# Convert to numpy arrays if needed
X_proper = to_numpy_safe(X_proper)
y_proper = to_numpy_safe(y_proper)
X_test   = to_numpy_safe(X_test)
y_test   = to_numpy_safe(y_test)

,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,...,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18,Feature_19,Target
1907,66.358060,1042.595014,0.416862,9.759544,0.812876,0.170636,0.745247,0.180941,0.665594,0.642942,...,0.702691,0.856997,0.113372,0.438890,0.829928,0.874499,0.881285,0.746344,0.162515,0.249900
1836,12.164998,574.252129,0.208135,1.643181,0.431878,0.414141,0.972406,0.897310,0.139641,0.979398,...,0.028695,0.265060,0.335338,0.163979,0.280915,0.607862,0.730891,0.405235,0.768671,0.067587
9843,66.008198,1277.479355,0.485108,2.093126,0.896072,0.154454,0.992653,0.214989,0.987448,0.951662,...,0.566142,0.692757,0.281621,0.075951,0.736105,0.498902,0.915329,0.708411,0.960809,0.354656
5340,84.886851,1013.863640,0.148065,2.256299,0.661381,0.315797,0.423807,0.148431,0.429247,0.744401,...,0.769733,0.133498,0.669849,0.886129,0.050140,0.964052,0.253382,0.540414,0.892665,0.097398
7901,41.417166,809.436576,0.600055,7.808032,0.975198,0.144761,0.262559,0.006367,0.913676,0.332045,...,0.101798,0.339526,0.045324,0.643148,0.803345,0.353261,0.952525,0.644390,0.284790,0.277290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3917,78.003049,1103.288571,0.591527,8.566431,0.535938,0.509949,0.205783,0.075388,0.833130,0.087189,...,0.969454,0.035090,0.417692,0.668971,0.720740,0.624277,0.924572,0.913075,0.092877,0.374134
2854,62.072633,469.051915,0.825910,4.406076,0.144357,0.407354,0.774754,0.943222,0.402089,0.693886,...,0.293584,0.232932,0.568222,0.311070,0.682635,0.467264,0.383137,0.993678,0.945827,0.223018
1080,57.629681,756.533935,0.563832,2.092401,0.951235,0.115585,0.681441,0.828510,0.597194,0.070250,...,0.488483,0.513389,0.125041,0.369754,0.619189,0.415613,0.307003,0.348084,0.260092,0.244684
8401,4.557764,1659.168426,0.651357,7.935800,0.927404,0.180993,0.583913,0.032631,0.655918,0.516134,...,0.605302,0.208553,0.581784,0.767043,0.370269,0.851475,0.773511,0.961745,0.616207,0.615900


In [5]:
from online_cp.CPS import NearestNeighboursPredictionMachine
from online_cp.CPS import RidgePredictionMachine
import numpy as np

cps = NearestNeighboursPredictionMachine(k=5)
# cps = RidgePredictionMachine(autotune=True)
cps.learn_initial_training_set(X_proper, y_proper)


# x = [7.34768661e+01, 
#      3.07355747e+02, 
#      9.98483471e-01, 
#      9.49902854e+00,
#      3.62115400e-01, 
#      5.48508689e-01, 
#      8.40112768e-01, 
#      3.42234629e-01,
#      3.44227224e-01, 
#      7.71883283e-01, 
#      2.42761017e-01, 
#      3.44293874e-01,
#      5.20936857e-01, 
#      5.20083931e-01, 
#      8.81639709e-01, 
#      3.41171486e-01,
#      1.60554105e-01, 
#      5.98680164e-01, 
#      7.88717442e-01, 
#      7.44306979e-02]

for i, (x, y) in enumerate(zip(X_test, y_test)):
    #try:

    cpd, precomputed = cps.predict_cpd(x=x, return_update=True)
    
    # except Exception as e:
    #     print(i)
    #     print()
    #     print("Exception occurred for x =", x)
    #     print()
    #     print("Error:", e)
    

[0.03626217167645584, 0.08284064528387171, 0.13063078461775385, 0.27238292527713465, 0.3809549718992853, 0.40944858751473073, 0.42123992344763883]
[0.036262171676455845, 0.08284064528387172, 0.13063078461775388, 0.2723829252771347, 0.3809549718992854, 0.40944858751473084, 0.42123992344763894]
[0.03626217167645583, 0.0828406452838717, 0.13063078461775382, 0.2723829252771346, 0.3809549718992852, 0.4094485875147306, 0.4212399234476387]
[0.2285166857539429, 0.31551885329723933, 0.42168032463201516, 0.4488487737475855, 0.515902646890535, 0.7326624413583925]
[0.22851668575394296, 0.3155188532972394, 0.42168032463201527, 0.44884877374758564, 0.5159026468905351, 0.7326624413583926]
[0.22851668575394285, 0.3155188532972393, 0.42168032463201505, 0.4488487737475854, 0.5159026468905349, 0.7326624413583924]
[0.05883713517733286, 0.08412438195581741, 0.10149829177708453, 0.10409835697237826, 0.10581814502275065, 0.10938996247518189]
[0.058837135177332876, 0.08412438195581742, 0.10149829177708455, 0.

/Users/simon/Documents/GitHub/Conformal-Predictive-Decision-Making/.venv/lib/python3.13/site-packages/online_cp/CPS.py:697: RuntimeWarning: invalid value encountered in add
  self.y_vals = np.array(sorted([-np.inf, np.inf] + self.Y[1: -1].tolist() + (self.Y[1: -1] + MACHINE_EPSILON(self.Y[1: -1])).tolist() + (self.Y[1: -1] - MACHINE_EPSILON(self.Y[1: -1])).tolist()))
/Users/simon/Documents/GitHub/Conformal-Predictive-Decision-Making/.venv/lib/python3.13/site-packages/online_cp/CPS.py:700: RuntimeWarning: invalid value encountered in add
  print((self.Y[1: -1] + MACHINE_EPSILON(self.Y[1: -1])).tolist())


ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
MACHINE_EPSILON = lambda x: np.abs(x) * np.finfo(np.float64).eps


print(np.inf + MACHINE_EPSILON(np.inf)))